# Implémentation d'un système expert animalier

Un exemple du [curriculum AI for Beginners](http://github.com/microsoft/ai-for-beginners).

Dans cet exemple, nous allons implémenter un système simple basé sur la connaissance pour déterminer un animal en fonction de certaines caractéristiques physiques. Le système peut être représenté par l'arbre AND-OR suivant (il s'agit d'une partie de l'arbre complet, nous pouvons facilement ajouter d'autres règles) :

![](../../../../../../translated_images/fr/AND-OR-Tree.5592d2c70187f283.webp)


## Notre propre shell de systèmes experts avec inférence rétrograde

Essayons de définir un langage simple pour la représentation des connaissances basé sur des règles de production. Nous utiliserons des classes Python comme mots-clés pour définir les règles. Il y aurait essentiellement 3 types de classes :
* `Ask` représente une question qui doit être posée à l'utilisateur. Elle contient l'ensemble des réponses possibles.
* `If` représente une règle, et ce n'est qu'un sucre syntaxique pour stocker le contenu de la règle
* `AND`/`OR` sont des classes pour représenter les branches ET/OU de l'arbre. Elles stockent simplement la liste des arguments à l'intérieur. Pour simplifier le code, toute la fonctionnalité est définie dans la classe parente `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Dans notre système, la mémoire de travail contiendrait la liste des **faits** sous forme de **paires attribut-valeur**. La base de connaissances peut être définie comme un grand dictionnaire qui associe des actions (nouveaux faits devant être insérés dans la mémoire de travail) à des conditions, exprimées sous forme d'expressions ET-OU. De plus, certains faits peuvent être `Ask`-és.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Pour effectuer l'inférence à rebours, nous définirons la classe `Knowledgebase`. Elle contiendra :
* Une `memory` de travail - un dictionnaire qui associe des attributs à des valeurs
* Des `rules` de la base de connaissances au format défini ci-dessus

Les deux méthodes principales sont :
* `get` pour obtenir la valeur d'un attribut, en effectuant l'inférence si nécessaire. Par exemple, `get('color')` obtiendra la valeur d'un emplacement de couleur (elle demandera si nécessaire, et stockera la valeur pour une utilisation ultérieure dans la mémoire de travail). Si l'on demande `get('color:blue')`, elle demandera une couleur, puis retournera une valeur `y`/`n` selon la couleur.
* `eval` effectue l'inférence réelle, c’est-à-dire qu’elle parcourt l’arbre AND/OR, évalue les sous-objectifs, etc.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Définissons maintenant notre base de connaissances sur les animaux et effectuons la consultation. Notez que cet appel vous posera des questions. Vous pouvez répondre en tapant `y`/`n` pour les questions oui-non, ou en spécifiant un numéro (0..N) pour les questions avec des réponses à choix multiples plus longues.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Utilisation d’Experta pour l’inférence avant

Dans l’exemple suivant, nous allons essayer de mettre en œuvre l’inférence avant en utilisant l’une des bibliothèques pour la représentation des connaissances, [Experta](https://github.com/nilp0inter/experta). **Experta** est une bibliothèque pour créer des systèmes d’inférence avant en Python, conçue pour être similaire au système classique ancien [CLIPS](http://www.clipsrules.net/index.html).

Nous aurions aussi pu implémenter nous-mêmes un chaînage avant sans trop de problèmes, mais les implémentations naïves ne sont généralement pas très efficaces. Pour un appariement plus efficace des règles, un algorithme spécial [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) est utilisé.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Nous allons définir notre système comme une classe qui hérite de `KnowledgeEngine`. Chaque règle est définie par une fonction distincte avec l'annotation `@Rule`, qui spécifie quand la règle doit être déclenchée. À l'intérieur de la règle, nous pouvons ajouter de nouveaux faits en utilisant la fonction `declare`, et l'ajout de ces faits entraînera l'appel de certaines règles supplémentaires par le moteur d'inférence avant.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Une fois que nous avons défini une base de connaissances, nous remplissons notre mémoire de travail avec quelques faits initiaux, puis appelons la méthode `run()` pour effectuer l'inférence. Vous pouvez voir en résultat que de nouveaux faits inférés sont ajoutés à la mémoire de travail, y compris le fait final concernant l'animal (si nous avons correctement configuré tous les faits initiaux).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Avertissement** :  
Ce document a été traduit à l’aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforçons d’assurer l’exactitude, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue native doit être considéré comme la source officielle. Pour les informations critiques, une traduction professionnelle réalisée par un humain est recommandée. Nous déclinons toute responsabilité en cas de malentendus ou d’interprétations erronées résultant de l’utilisation de cette traduction.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
